In [12]:
!git clone https://github.com/qubvel/efficientnet.git

fatal: destination path 'efficientnet' already exists and is not an empty directory.


In [26]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imutils import paths

input_shape = (300, 300, 3)
BATCH_SIZE = 32


trainPath = os.path.sep.join(['dataset/dfdc' ,"training"])
valPath = os.path.sep.join(['dataset/dfdc' ,"validation"])
testPath = os.path.sep.join(['dataset/dfdc', "evaluation"])

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=3,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=False,
	fill_mode="nearest")
# trainAug = ImageDataGenerator()
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="binary",
	target_size=input_shape,
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="binary",
	target_size=input_shape,
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="binary",
	target_size=input_shape,
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

labels = (trainGen.class_indices)
print(labels)

Found 149553 images belonging to 2 classes.
Found 18691 images belonging to 2 classes.
Found 18705 images belonging to 2 classes.
{'FAKE': 0, 'REAL': 1}


In [27]:
# models can be build with Keras or Tensorflow frameworks
# use keras and tfkeras modules respectively
# efficientnet.keras / efficientnet.tfkeras
import os
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
sys.path.append('..')

from keras.applications.imagenet_utils import decode_predictions
from efficientnet.efficientnet.tfkeras import EfficientNetB6
from efficientnet.efficientnet.tfkeras import center_crop_and_resize, preprocess_input

model = EfficientNetB6(weights='noisy-student', include_top=False) # or weights='noisy-student'
for layer in model.layers:
	layer.trainable = False
for layer in model.layers[50:]:
	layer.trainable = True
model.summary()
print('len layers: ' , len(model.layers))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 5 1512        input_8[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 5 224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 5 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

len layers:  659


In [28]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
# from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.optimizers import adam


x = model.get_layer('top_activation').output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid', name='lastfc')(x)
efficinet_model = Model(model.input, out)

efficinet_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False),metrics=['accuracy'])

checkpoint = ModelCheckpoint('./weights.{epoch:02d}-{val_loss:.2f}.model',save_weights_only=False, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, Y_val), verbose = 1)
efficinet_model.summary()
H = efficinet_model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=totalVal // BATCH_SIZE,
	epochs=20, verbose = 1, callbacks=callbacks_list)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 5 1512        input_8[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 5 224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 5 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

Epoch 1/20


StopIteration: could not broadcast input array from shape (300,300,3) into shape (300,300,3,3)